# Predictive Insurance Agent with KumoRFM

This notebook demonstrates how to leverage the `kumo-rfm-mcp` (https://github.com/kumo-ai/kumo-rfm-mcp) to build agents which make predictions (forecasts, risk scores, etc.) as part of 
their toolkit. We demonstrate the utility of predictions with a simple *insurance agent*, which examines expiring policies and suggests optimal bundling and reachout strategies.

The example uses `claude-code-sdk-python` (https://github.com/anthropics/claude-code-sdk-python) to orchestrate the agentic flow.

### Prerequisites
Get your `KUMO_API_KEY` for free at https://kumorfm.ai/, make sure to store it as an environment variable before running this notebook.
```
export KUMO_API_KEY = <your-kumorfm-api-key>
```
Make sure to follow the installation steps below. 

### Installation

In [1]:
!uv pip install claude-code-sdk
!uv pip install kumo-rfm-mcp
!uv pip install fsspec s3fs
# make sure claude code is installed: @anthropic-ai/claude-code from NPM
# claude code requires Node.js 18+ (https://docs.anthropic.com/en/docs/claude-code/sdk/sdk-python#prerequisites)
!npm install -g @anthropic-ai/claude-code

Using Python 3.11.13 environment at: /Users/blazstojanovic/Developer/anth-cookbooks-env
Audited 1 package in 21ms
Using Python 3.11.13 environment at: /Users/blazstojanovic/Developer/anth-cookbooks-env
Audited 1 package in 15ms
Using Python 3.11.13 environment at: /Users/blazstojanovic/Developer/anth-cookbooks-env
Audited 2 packages in 4ms
⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 9 packages, and changed 3 packages in 1s
⠏
⠏11 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [2]:
import asyncio
import json
import os

from claude_code_sdk import (
    AssistantMessage,
    ClaudeCodeOptions,
    ClaudeSDKClient,
    ResultMessage,
    SystemMessage,
    TextBlock,
    ThinkingBlock,
    ToolResultBlock,
    ToolUseBlock,
    UserMessage,
)

## Configuring the KumoRFM MCP server

In [ ]:
KUMO_API_KEY = os.getenv("KUMO_API_KEY")
mcp_servers = {
    # KumoRFM MCP
    "kumo-rfm": {
        "command": "python3",
        "args": ["-m", "kumo_rfm_mcp.server"],
        "env": {"KUMO_API_KEY": f"{KUMO_API_KEY}"},
    },
}

## Agent Prompt
Let's define a simple prompt which will characterize our insurance agent. We want to keep the prompt simple and let the underlying intelligence of the LLM shine through. This example is especially simple but much more complicated flows can be constructed if needed.

In [4]:
# Prompt
PROMPT = """You are a insurance company agent. You will use KumoRFM MCP to obtain insight about customers and provide expert guidance to the user.

Query expectations:
* The query should contain a data path and a list of filenames
* The query should contain a set of entities it concerns (e.g. customer IDs)

Your role:
1. Assist the user by making the necessary predictions and using them to resolve the query as well as possible
2. Using KumoRFM:
    - Inspect the data with the appropriate tool
    - Use your common sense to determine how the graph should be structured
    - Explain what graph structure you have choesen and why
    - Set up the graph metadata
    - Materialize the graph
    - Make sure to check documentation of predictive query language before predicting!
    - Make predictions (make sure to run in best mode!)
3. Provide expert support to the customer, only make the predictions the customer asks for!
"""

## Query Data

Define the sample query with customer data and IDs for analysis.


In [5]:
QUERY = """
Hi! I'm working in customer support and I have a list of customers which have expiring policies in 30 days. Please find out which
users are most likely to churn on their policies. For the three customers most likely to churn, recomend which policy they
are most likely to purchase next. Finally, help me write promotion emails bundling policies together.
Output a brief summary of your findings, with the proposed emails.

Data Path: s3://kumo-sdk-public/rfm-datasets/insurance/
File Names: policies.csv, customers.csv, products.csv
Customer IDs: [13, 402, 14, 329, 375]
"""

# Choose from File Names: agents.csv, policies.csv, customers.csv, pets.csv, products.csv, properties.csv, vehicles.csv
# Choose from example customers: [13, 402, 14, 329, 375, 239, 493, 319]

## Main Analysis Function
Main function, which creates a `ClaudeSDKClient` with the `kumo-rfm-mcp` server and runs the query above!

In [9]:
from pprint import pprint


def display_message(msg):
    """Display message content in a clean format."""
    if isinstance(msg, UserMessage):
        for block in msg.content:
            if isinstance(block, TextBlock):
                pprint(f"User: {block.text}")
            elif isinstance(block, ToolResultBlock):
                pprint(f"Tool Result: {block.content if block.content else 'None'}")
    elif isinstance(msg, AssistantMessage):
        for block in msg.content:
            if isinstance(block, TextBlock):
                pprint(f"Claude: {block.text}")
            elif isinstance(block, ToolUseBlock):
                pprint(f"Using tool: {block.name}")
                # Show tool inputs
                if block.input:
                    print(f"  Input: {block.input}")
    elif isinstance(msg, SystemMessage):
        # Ignore system messages
        pass
    elif isinstance(msg, ResultMessage):
        print("Result ended")
        if msg.total_cost_usd:
            pprint(f"Cost: ${msg.total_cost_usd:.6f}")


async def main(prompt: str, query: str, max_turns: int = 35):
    """Run insurance analysis using KumoRFM MCP server with streaming client."""
    print(f"\n{'=' * 50}")
    print("Starting Insurance Analysis")
    print(f"{'=' * 50}")

    async with ClaudeSDKClient(
        options=ClaudeCodeOptions(
            mcp_servers=mcp_servers,
            system_prompt=prompt,
            model="claude-3-7-sonnet-latest",
            allowed_tools=[
                "mcp__kumo-rfm__get_docs",
                "mcp__kumo-rfm__authenticate",
                "mcp__kumo-rfm__find_table_files",
                "mcp__kumo-rfm__inspect_table_files",
                "mcp__kumo-rfm__inspect_graph_metadata",
                "mcp__kumo-rfm__update_graph_metadata",
                "mcp__kumo-rfm__get_mermaid",
                "mcp__kumo-rfm__materialize_graph",
                "mcp__kumo-rfm__lookup_table_rows",
                "mcp__kumo-rfm__predict",
                "mcp__kumo-rfm__evaluate",
                "mcp__kumo-rfm__list_mcp_resources",
                "mcp__kumo-rfm__read_mcp_resource",
            ],
            max_turns=25,
        )
    ) as client:
        await client.query(query)

        # Stream responses with clear formatting
        async for message in client.receive_response():
            display_message(message)

## Run the Analysis

Execute the insurance agent analysis. Make sure you have set your `KUMO_API_KEY` environment variable before running this cell.


In [8]:
# Run as script
await main(prompt=PROMPT, query=QUERY)


Starting Insurance Analysis
("Claude: I'll help you analyze which customers with expiring policies are "
 'most likely to churn, recommend potential new policies for the three '
 'highest-risk customers, and create promotion emails bundling policies '
 "together. Let's use KumoRFM to gain insights and make predictions.\n"
 '\n'
 "First, let's authenticate with KumoRFM and inspect the data.")
'Using tool: mcp__kumo-rfm__authenticate'
("Tool Result: [{'type': 'text', 'text': 'KumoRFM session successfully "
 "authenticated'}]")
("Claude: Now, let's inspect the provided dataset files to understand the data "
 "we're working with:")
'Using tool: mcp__kumo-rfm__inspect_table_files'
  Input: {'paths': ['s3://kumo-sdk-public/rfm-datasets/insurance/policies.csv', 's3://kumo-sdk-public/rfm-datasets/insurance/customers.csv', 's3://kumo-sdk-public/rfm-datasets/insurance/products.csv'], 'num_rows': 5}
("Tool Result: [{'type': 'text', 'text': "
 '\'{"s3://kumo-sdk-public/rfm-datasets/insurance/poli